In [ ]:
%pip install google-genai

In [6]:
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
import os
import json
from google import genai
from google.genai.types import Content, GenerationConfig, GenerateContentConfig # <-- Import the parent config class
from dotenv import load_dotenv

In [8]:
#1. Setup
# Assumes GEMINI_API_KEY is set in your environment

# Load variables from the .env file in the current directory
load_dotenv()

# The client automatically finds and uses the key
client = genai.Client()
print("Gemini Client initialized successfully using key from .env file.")

MODEL_NAME = 'gemini-2.5-flash'
CONTEXT_FILE_PATH = "fat_lab.json" # <-- Using the JSON file name

Gemini Client initialized successfully using key from .env file.


In [20]:
#2. Load and Parse the JSON Context
def load_and_parse_json(file_path):
    """
    Reads a JSON file containing a list of Q&A entries and converts it 
    into a single, structured text block for the LLM context.
    """
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            data = json.load(f)
    except FileNotFoundError:
        print(f"Error: Context file not found at '{file_path}'.")
        return None
    except json.JSONDecodeError:
        print(f"Error: Could not decode JSON from '{file_path}'. Ensure the file is valid JSON.")
        return None
    except Exception as e:
        print(f"An error occurred while reading the file: {e}")
        return None

    context_parts = []
    
    # Iterate through each Q&A object in the JSON list
    for entry in data:
        speaker = entry.get("speaker", "Unknown")
        topic = entry.get("topic", "General")
        question = entry.get("question", "")
        answer = entry.get("answer", "")
        tags = ", ".join(entry.get("tags", []))
        
        # Format the entry into a single text block
        chunk = (
            f"Speaker: {speaker}\n"
            f"Topic: {topic}\n"
            f"Question: {question}\n"
            f"Answer: {answer}\n"
            f"Tags: {tags}\n"
        )
        context_parts.append(chunk)

    # Join all formatted chunks with two newlines for separation
    return "\n\n".join(context_parts)

full_interview_context = load_and_parse_json(CONTEXT_FILE_PATH)

if not full_interview_context:
    exit() # Stop execution if context couldn't be loaded

In [21]:
#3. Construct the Prompt
SYSTEM_INSTRUCTION = f"""
You are a useful chatbot, built to answer questions that cafe entreprenuers when starting a new cafe.
Use the following interview transcript context to answer the user's request.
If the answer is not contained in the context, say you don't know.

--- INTERVIEW CONTEXT START ---
{full_interview_context}
--- INTERVIEW CONTEXT END ---
"""


In [22]:
#4. User Query
user_query = "What percentage of the budget is allocated to marketing, and what was the gap they noticed in the market?"

In [30]:
# 5. Generate Content

# Prepare contents list
contents = [
    # Message 1: The System Instruction + Full Context
    Content(role='user', parts=[genai.types.Part.from_text(text=SYSTEM_INSTRUCTION)]),
    # Message 2: The actual User Question
    Content(role='user', parts=[genai.types.Part.from_text(text=user_query)])
]

# FIX: Combine GenerationConfig and GenerateContentConfig into one call.
# This structure satisfies the requirement of the Pydantic validation 
# in your current SDK version by passing the parameter directly.
full_config = GenerateContentConfig(
    temperature=0.0
)

print(f"User Query: {user_query}\n")
print(f"Loading context from: {CONTEXT_FILE_PATH} (Length: {len(full_interview_context)} characters)")
print("--- Generating response with Gemini ---")

# --- Final API Call ---
response = client.models.generate_content(
    model=MODEL_NAME,
    contents=contents,
    config=full_config 
)

print("\nChatbot Response:")
print(response.text)

User Query: What percentage of the budget is allocated to marketing, and what was the gap they noticed in the market?

Loading context from: fat_lab.json (Length: 7581 characters)
--- Generating response with Gemini ---

Chatbot Response:
Around **1.5–2% of their revenue** is allocated to direct marketing and agencies.

The market gap they noticed was a **lack of cafes serving good filter coffee and omelettes in Pune**. This niche inspired their initial limited menu.

Chatbot Response:
Around **1.5–2% of their revenue** is allocated to direct marketing and agencies.

The market gap they noticed was a **lack of cafes serving good filter coffee and omelettes in Pune**. This niche inspired their initial limited menu.
